We now demonstrate using the `fastai` convenience method to perform the collaborative filtering task using the neural network model. Here, instead of a single FC layer of 100 nodes, we use 2 hidden FC layers; one with 100 nodes and the second with 50 nodes. There are additional some architectural differences to help with regularizing deep models.

In [1]:
from fastai.collab import *
from fastai.tabular.all import *

# Dataset preparation
path = untar_data(URLs.ML_100k)

ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0, 1), names=('movie_id', 'title'), header=None)
ratings = ratings.merge(movies)
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)

# model and learner instantiation
learn = collab_learner(dls, use_nn=True, y_range=(0, 5.5), layers=[100, 50])

# training
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.969874,0.978763,00:08
1,0.901125,0.920941,00:08
2,0.840122,0.891458,00:08
3,0.741204,0.868519,00:08
4,0.749001,0.861849,00:08


We discuss more on the architectural differences. `collab_learner` with `use_nn=True` uses an `EmbeddingNN` model, which just delegates to `TabularModel` with the additional specification that there are no continuous variables in the input, and that the output is one-dimensional.

In [2]:
learn.model

EmbeddingNN(
  (embeds): ModuleList(
    (0): Embedding(944, 74)
    (1): Embedding(1665, 102)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): Linear(in_features=176, out_features=100, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): LinBnDrop(
      (0): Linear(in_features=100, out_features=50, bias=False)
      (1): ReLU(inplace=True)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=50, out_features=1, bias=True)
    )
    (3): fastai.layers.SigmoidRange(low=0, high=5.5)
  )
)

Inspecteng the model, first observe that `emb_dropout` does nothing (no dropout, since `p=0`), and theat `bn_cont` does nothing (it regularizes the continuous variable inputs, for tabular data). The major difference apart fron there being two hidden linear layers now is that `BatchNorm1d` is used to provide some regularization.